# Baseline

In [1]:
import torch
from PIL import Image
import torchvision.transforms as T
from torch import nn
import numpy as np
from tqdm import tqdm
import random
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
input_dir = "/kaggle/input/blood-vessel-segmentation/"
train_dir = input_dir + "train"

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)


print(device)

cuda


## Load data

**Datasets**

- kidney_1_dense	
    - images
    - labels
- kidney_1_voi  
    - images
    - labels
- kidney_2	
    - images
    - labels
- kidney_3_dense	
    - labels
- kidney_3_sparse
    - images
    - labels

### ¿Do I use one dataset or all of them? ¿How do I merge all the datasets?

I think i can merge the datasets by just merging the images and labels paths. But for now I will just use the `kidney_1_dense` dataset.

### Dataset

In [2]:
class KidneyDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, msk_paths, transform=None, target_transform=None):
        self.img_paths  = img_paths
        self.msk_paths  = msk_paths
        self.transform = transform
        self.target_transform = target_transform
        assert len(self.img_paths) == len(self.msk_paths)
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        msk_path = self.msk_paths[idx]

        img = Image.open(img_path)
        msk = Image.open(msk_path)

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            msk = self.target_transform(msk).squeeze().float()

        return img, msk

In [3]:
k1d_path = "kidney_1_dense"
h_resize, w_resize = 512, 512

folder_path = os.path.join(train_dir, k1d_path)
k1d_imgs_paths = sorted(os.path.join(folder_path, "images", file_name) for file_name in os.listdir(os.path.join(folder_path, "images")))
k1d_msks_paths = sorted(os.path.join(folder_path, "labels", file_name) for file_name in os.listdir(os.path.join(folder_path, "images")))

transform = T.Compose([
    T.ToTensor(),  
    T.ConvertImageDtype(torch.float32),
    T.Resize((h_resize, w_resize), antialias=True)
])

target_transform = T.Compose([
    T.PILToTensor(), 
    T.Resize((h_resize, w_resize), antialias=True)
])

k1d_ds = KidneyDataset(k1d_imgs_paths, k1d_msks_paths, transform=transform, target_transform=target_transform)

In [4]:
g = torch.Generator().manual_seed(42)

train_ds, eval_ds = torch.utils.data.random_split(
    k1d_ds,
    [0.8, 0.2],
    generator=g
)

bs = 32
num_workers = 2
train_dl = DataLoader(train_ds, batch_size=bs, num_workers=num_workers, shuffle=True)
eval_dl = DataLoader(eval_ds, batch_size=bs, num_workers=num_workers, shuffle=False)

In [5]:
len(train_ds), len(eval_ds)

(1824, 455)

## Define model

In [6]:
class SegmentationModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SegmentationModel, self).__init__()

        # Define the encoder - a series of convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Define the decoder - a series of transposed convolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, num_classes, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the model with the desired input channels and number of classes
net = SegmentationModel(in_channels=1, num_classes=1)
net.to(device)

SegmentationModel(
  (encoder): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 64, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

## Define evaluation metric

In [7]:
class IouMetric:
    def __init__(self, num_classes: int, int2str: dict, ignore_index: int = -1, prefix="train"):
        """
        Args:
            num_classes: number of classes
            int2str: dictionary mapping class index to class name
            ignore_index: index to ignore in the metric calculations
            prefix: prefix to use for logging
        """
        self.area_intersect = torch.zeros(num_classes)
        self.area_label = torch.zeros(num_classes)
        self.area_pred = torch.zeros(num_classes)
        self.num_classes = num_classes
        self.int2str = int2str
        self.ignore_index = ignore_index
        self.prefix = prefix

    def process(self, preds, labels):
        mask = labels != self.ignore_index
        preds = preds[mask]
        labels = labels[mask]

        # compute area of intersection, label and prediction
        intersect = preds[preds == labels]
        area_intersect = torch.histc(intersect.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)
        area_label = torch.histc(labels.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)
        area_pred = torch.histc(preds.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)

        # update results
        self.area_intersect += area_intersect.cpu()
        self.area_label += area_label.cpu()
        self.area_pred += area_pred.cpu()

    def compute(self) -> dict:
        iou = self.area_intersect / (self.area_label + self.area_pred - self.area_intersect)
        if 0 <= self.ignore_index <= self.num_classes - 1:
            iou[self.ignore_index] = torch.nan
        mean_iou = torch.nanmean(iou)

        metrics = {
            f"{self.prefix}/iou_{self.int2str[idx]}": iou[idx].item()
            for idx in range(len(iou))
            if idx != self.ignore_index
        }
        metrics[f"{self.prefix}/mean_iou"] = mean_iou.item()

        return metrics

    def reset(self):
        self.area_intersect.zero_()
        self.area_label.zero_()
        self.area_pred.zero_()


## Define loss function


In [8]:
loss_fn = nn.BCEWithLogitsLoss()

## Train model

### Optimizer and scheduler

In [9]:
lr = 1e-5
optimizer = torch.optim.Adam(lr=lr, params=net.parameters())

### Train method

In [10]:
wandb_log = False
# log training and data config
if wandb_log:
    wandb.login(key=wandb_key)
    wandb.init(
        project="blood-vessel-segmentation",
        config=dict(
            optimizer=type(optimizer).__name__,
            loss_fn=type(loss_fn).__name__,
        ),
    )
# metrics
train_iou = IouMetric(num_classes=2, int2str=["batckground", "Vessel"], ignore_index=0)
eval_iou = IouMetric(num_classes=2, int2str=["batckground", "Vessel"], ignore_index=0, prefix="eval")

def train():
    train_loss = 0.0
    net.train()
    for x, y in tqdm(train_dl):
        x, y = x.to(device), y.to(device)

        logits = net(x).squeeze()
        loss = loss_fn(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = logits.sigmoid().round().detach()
        train_iou.process(preds, y)

        train_loss += loss.item()

    train_loss /= len(train_dl)

    metrics_dict = train_iou.compute()
    print({"epoch": epoch, "train/mean_loss": train_loss, **metrics_dict})

    if wandb_log:
        wandb.log({"epoch": epoch, "train/mean_loss": train_loss, **metrics_dict})
        train_iou.reset()

In [11]:
def eval():
    eval_loss = 0.0
    net.eval()
    for x, y in tqdm(eval_dl):
        with torch.no_grad():
            x, y = x.to(device), y.to(device)
            logits = net(x).squeeze()
            loss = loss_fn(logits, y)

        preds = logits.sigmoid().round()
        eval_iou.process(preds, y)

        eval_loss += loss.item()

    eval_loss /= len(eval_dl)

    metrics_dict = eval_iou.compute()
    print({"epoch": epoch, "val/mean_loss": eval_loss, **metrics_dict})
    if wandb_log:
        wandb.log({"epoch": epoch, "val/mean_loss": val_loss, **metrics_dict})
    eval_iou.reset()



In [12]:
epochs = 3
for epoch in range(epochs):
    print("Epoch", epoch)
    train()
    eval()

if wandb_log:
    wandb.finish()


Epoch 0


100%|██████████| 57/57 [00:39<00:00,  1.43it/s]


{'epoch': 0, 'train/mean_loss': 0.652239061238473, 'train/iou_Vessel': 0.023031318560242653, 'train/mean_iou': 0.023031318560242653}


100%|██████████| 15/15 [00:10<00:00,  1.36it/s]


{'epoch': 0, 'val/mean_loss': 0.6412355224291484, 'eval/iou_Vessel': 0.023709958419203758, 'eval/mean_iou': 0.023709958419203758}
Epoch 1


...

{'epoch': 1, 'val/mean_loss': 0.6077883203824361, 'eval/iou_Vessel': 0.023709958419203758, 'eval/mean_iou': 0.023709958419203758}
Epoch 2


100%|██████████| 57/57 [00:38<00:00,  1.50it/s]


{'epoch': 2, 'train/mean_loss': 0.5754592585981938, 'train/iou_Vessel': 0.023100342601537704, 'train/mean_iou': 0.023100342601537704}


100%|██████████| 15/15 [00:05<00:00,  2.72it/s]

{'epoch': 2, 'val/mean_loss': 0.5633396744728089, 'eval/iou_Vessel': 0.023709958419203758, 'eval/mean_iou': 0.023709958419203758}


### Evaluation method

## Make submisssion